<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />

<br><h1>Building a Classification Model in Kaggle</h1>

<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />

<br>
<h2>Purpose of this Script</h2>
This script is designed to demonstrate how to develop a model in Kaggle.
<br><br><br>
<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

Below is a starter code that comes with Jupyter Notebook.

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/machine-learning-nycu-final-project-2023/sample_submission.csv
/kaggle/input/machine-learning-nycu-final-project-2023/train.csv
/kaggle/input/machine-learning-nycu-final-project-2023/test.csv
/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br><br>
You will need to import any additional libraries. It's also a good idea to get rid of the cell above once you're used to Jupyter Notebook in Kaggle.<br>

In [3]:
import numpy as np
import pandas as pd 
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, mean_squared_error, r2_score, roc_auc_score, roc_curve, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
from sklearn.impute import SimpleImputer

In [4]:
# modeling library
import sklearn.linear_model                          # linear modeling in scikit-learn

# other model building tools
from sklearn.model_selection import train_test_split # train-test split
from sklearn.metrics import roc_auc_score            # auc score

Make sure to tell Python the path to your training data.

In [5]:
# importing the training dataset
path             = "/kaggle/input/spaceship-titanic/"
training_dataset = "train.csv"


# reading in the .csv file with pandas
titanic_train    = pd.read_csv(filepath_or_buffer = path + training_dataset)


# checking basic info about the dataset
titanic_train.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>
The test set is also available. Notice that there is no data for Transported (y-variable) in the test set. This is intentional.<br>

In [6]:
# importing the training dataset
path             = "/kaggle/input/spaceship-titanic/"
testing_dataset  = 'test.csv'

# importing the testing dataset
titanic_test = pd.read_csv(filepath_or_buffer = path + testing_dataset)

# checking basic info about the dataset
titanic_test.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   4277 non-null   object 
 1   HomePlanet    4190 non-null   object 
 2   CryoSleep     4184 non-null   object 
 3   Cabin         4177 non-null   object 
 4   Destination   4185 non-null   object 
 5   Age           4186 non-null   float64
 6   VIP           4184 non-null   object 
 7   RoomService   4195 non-null   float64
 8   FoodCourt     4171 non-null   float64
 9   ShoppingMall  4179 non-null   float64
 10  Spa           4176 non-null   float64
 11  VRDeck        4197 non-null   float64
 12  Name          4183 non-null   object 
dtypes: float64(6), object(7)
memory usage: 434.5+ KB


<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>
You may want to join the datasets together before feature engineering. This will help make things more efficient.<br>

In [7]:
titanic_train['set'] = 'Training'
titanic_test ['set'] = 'Testing'

# concatenating both datasets together for mv and feature engineering
titanic_df = pd.concat([titanic_train , titanic_test])

# resetting index to avoid problems later in the code
titanic_df.reset_index(drop = False, inplace = True)
titanic_df_revise = titanic_df

In [8]:
## Groups of people have different survival rate.
titanic_df['Group'] = titanic_df['PassengerId'].astype(str).str[:4]

counts = titanic_df['Group'].value_counts()
titanic_df['Group'] = titanic_df['Group'].map(counts)

titanic_df.groupby('Group')['Transported'].mean()

Group
1    0.452445
2    0.538050
3    0.593137
4    0.640777
5    0.592453
6    0.614943
7    0.541126
8    0.394231
Name: Transported, dtype: float64

In [9]:
drop_col = ['index' , 'PassengerId' , 'Name']
titanic_df = titanic_df.drop(drop_col , axis = 1)

In [10]:
## Split the Cabin information into three different variable for analysis
titanic_df[['Deck', 'Number', 'Side']] = titanic_df['Cabin'].str.split('/', expand=True)
titanic_df = titanic_df.drop('Cabin' , axis = 1)
titanic_df = titanic_df.drop('Number' , axis = 1)

titanic_df.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,set,Group,Deck,Side
0,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False,Training,1,B,P
1,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True,Training,1,F,S
2,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False,Training,2,A,S
3,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False,Training,2,A,S
4,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True,Training,1,F,S


In [11]:
titanic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12970 entries, 0 to 12969
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    12682 non-null  object 
 1   CryoSleep     12660 non-null  object 
 2   Destination   12696 non-null  object 
 3   Age           12700 non-null  float64
 4   VIP           12674 non-null  object 
 5   RoomService   12707 non-null  float64
 6   FoodCourt     12681 non-null  float64
 7   ShoppingMall  12664 non-null  float64
 8   Spa           12686 non-null  float64
 9   VRDeck        12702 non-null  float64
 10  Transported   8693 non-null   object 
 11  set           12970 non-null  object 
 12  Group         12970 non-null  int64  
 13  Deck          12671 non-null  object 
 14  Side          12671 non-null  object 
dtypes: float64(6), int64(1), object(8)
memory usage: 1.5+ MB


In [12]:
Expense_column = ['RoomService' , 'FoodCourt' , 'ShoppingMall' , 'Spa' , 'VRDeck']
titanic_df['Expense'] = np.sum(titanic_df[Expense_column] , axis = 1)

In [13]:
titanic_df.loc[:,['CryoSleep']]=titanic_df.apply(lambda x: True if x.Expense == 0 and pd.isna(x.CryoSleep) else x,axis =1)

In [14]:
cat_col = ['HomePlanet' , 'Destination' , 'Deck' , 'Side' , 'CryoSleep' , "VIP"]
titanic_df.fillna(titanic_df[cat_col].mode().iloc[0], inplace=True)

num_col =  ['Age' , 'RoomService' , 'FoodCourt' , 'ShoppingMall' , 'Spa' , 'VRDeck']
titanic_df.fillna(titanic_df[num_col].mean().iloc[0], inplace=True)

In [15]:
titanic_df['Age_group'] = 0
titanic_df.loc[titanic_df['Age']  <= 5, 'Age_group'] = 1
titanic_df.loc[(titanic_df['Age'] > 5) & (titanic_df['Age'] <= 10), 'Age_group']=2
titanic_df.loc[(titanic_df['Age'] > 10) & (titanic_df['Age'] <= 20), 'Age_group']=3
titanic_df.loc[(titanic_df['Age'] > 20) & (titanic_df['Age'] <= 30), 'Age_group']=4
titanic_df.loc[(titanic_df['Age'] > 30) & (titanic_df['Age'] <= 50), 'Age_group']=5
titanic_df.loc[(titanic_df['Age'] > 50) & (titanic_df['Age'] <= 60), 'Age_group']=6
titanic_df.loc[(titanic_df['Age'] > 60) & (titanic_df['Age'] <= 70), 'Age_group']=7
titanic_df.loc[titanic_df['Age']  > 70, 'Age_group'] = 8

In [16]:
titanic_df = pd.get_dummies(titanic_df, columns = ['Destination', 'Deck', 'Side', 'HomePlanet'], drop_first=False )

In [17]:
titanic_df['CryoSleep'] = titanic_df['CryoSleep'].astype('int')

In [18]:
titanic_df['VIP'] = titanic_df['VIP'].astype('int')

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>
Let's look at correlations to find an x-variable to model with.<br>

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>
Since training and testing sets have already been developed for us, we can simply select the X-features we want to model with and we're good to go.<br>

In [19]:
titanic_df.head(); titanic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12970 entries, 0 to 12969
Data columns (total 29 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   CryoSleep                  12970 non-null  int64  
 1   Age                        12970 non-null  float64
 2   VIP                        12970 non-null  int64  
 3   RoomService                12970 non-null  float64
 4   FoodCourt                  12970 non-null  float64
 5   ShoppingMall               12970 non-null  float64
 6   Spa                        12970 non-null  float64
 7   VRDeck                     12970 non-null  float64
 8   Transported                12970 non-null  object 
 9   set                        12970 non-null  object 
 10  Group                      12970 non-null  int64  
 11  Expense                    12970 non-null  float64
 12  Age_group                  12970 non-null  int64  
 13  Destination_55 Cancri e    12970 non-null  uin

In [20]:
drop_list=['ShoppingMall',"Age",'HomePlanet_Earth','FoodCourt','Destination_TRAPPIST-1e', "Side_P" , 'Transported' , 'set']

In [21]:
# setting explanatory variable(s) with most correlated x-variable
x_train = titanic_df.drop(drop_list , axis = 1)[ titanic_df['set'] == 'Training']

# setting response variable
y_train = titanic_df[ 'Transported' ][ titanic_df['set']   == 'Training' ].astype(dtype = 'int')

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>
Even though there's already a testing set, it's a good idea to build one on the training data to assess model performance and stability qualities.<br>

In [22]:
# developing training and validation sets
x_train_1, x_train_2, y_train_1, y_train_2 = train_test_split(
            x_train,
            y_train.astype(dtype = 'int'),
            random_state = 50,
            test_size    = 0.2,
            stratify     = y_train)

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>
Below is a Logistic Regression model using scikit-learn. Note that as you explore additional scikit-learn models, the only things that need to change are the model name and the model type.<br>

In [23]:
kfold = StratifiedKFold(n_splits=10)
# Modeling step Test differents algorithms 
random_state = 2
classifiers = [SVC(),RandomForestClassifier(), ExtraTreesClassifier(),GradientBoostingClassifier(),\
    KNeighborsClassifier(), CatBoostClassifier(verbose = False),\
    XGBClassifier(), LGBMClassifier()]

cv_results = []
for classifier in classifiers :
    cv_results.append(cross_val_score(classifier, x_train, y = y_train, scoring = "accuracy", cv = kfold, n_jobs=4))

cv_means = []
cv_std = []
for cv_result in cv_results:
    cv_means.append(cv_result.mean())
    cv_std.append(cv_result.std())

cv_res = pd.DataFrame({"CrossValMeans":cv_means,"CrossValerrors": cv_std,"Algorithm":["SVC", "RandomForest","ExtraTrees","GradientBoosting"
                                                                                      ,"KNeighboors", "CatBoostClassifier","XGBClassifier", "LGBMClassifier"]})

In [24]:
cv_res

,CrossValMeans,CrossValerrors,Algorithm
0,0.789490,0.015817,SVC
1,0.796620,0.009012,RandomForest
2,0.772234,0.009455,ExtraTrees
3,0.796738,0.013818,GradientBoosting
4,0.760959,0.012693,KNeighboors
5,0.808702,0.015206,CatBoostClassifier
6,0.800074,0.011678,XGBClassifier
7,0.803411,0.014570,LGBMClassifier


In [31]:
model = CatBoostClassifier(verbose = False)
model.fit(x_train , y_train)

model.score(x_train, y_train).round(4)

0.8533

In [32]:
x_test = titanic_df.drop(drop_list , axis = 1)[ titanic_df['set'] == 'Testing']
model_pred = model.predict(x_test)

In [27]:
model = CatBoostClassifier(verbose = False)
params = {'learning_rate': np.arange(0.01, 0.5, 0.03), 'n_estimators': range(100, 500, 50), 'max_depth': [2, 3, 5]}

grid_clf = sklearn.model_selection.RandomizedSearchCV(model, param_distributions=params, cv=5).fit(x_train, y_train)
grid_clf.best_params_

{'n_estimators': 150, 'max_depth': 5, 'learning_rate': 0.16}

In [28]:
best_grid_clf = grid_clf.best_estimator_
best_grid_clf.score(x_train, y_train).round(4)

0.8379

In [30]:
x_test = titanic_df.drop(['Transported' , 'set' ] , axis = 1)[ titanic_df['set'] == 'Testing']
model_pred = best_grid_clf.predict(x_test)

In [33]:
# saving predictions with their respective Ids from the test set
predictions = pd.DataFrame(data = { 'PassengerId' : titanic_test['PassengerId'],
                                    'Transported' : model_pred.astype(bool)               } )

# checking the results
predictions.head(n = 5)

# sending predictions to .csv file
predictions.to_csv(path_or_buf = 'space_titanic_test01.csv',
                   index = False)

<br>